<a href="https://colab.research.google.com/github/vijay-lab/internship/blob/master/latest_Smriti_scrapper%20V%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 69.4 MB of archives.
After this operation, 254 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 75.0.3770.142-0ubuntu0.18.04.1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 75.0.377

In [0]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [0]:
#loading ther chrome driver for colab ## driver = webdriver.Chrome('chromedriver',options=options)

In [18]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 18 23:44:43 2019

@author: TAPAN
"""

import selenium
import requests
import numpy as np

from collections import OrderedDict
import pandas as pd
from selenium import webdriver


letters_val=['k','l','m','n','o','p','q','r','s','t','u','v','w','y','z']
# 'a','b','c','d','e','f','g','h','i','j',

columns=['movie','song','singers','lyricist','year','music_director','lyrics']
songs_df_main=pd.DataFrame()

# setting to run colab headless. 
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver = webdriver.Chrome("chromedriver",options=options)

from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,ElementNotVisibleException,ElementNotInteractableException

for letter in letters_val:
    print('currently scraping singers by letter',letter)
    address="http://smriti.com/hindi-songs/singers-"+letter  # selects singer letter 
    driver.get(address)
    #singers=[]
    songs_df_main=pd.DataFrame()
    for singer in range(2,1000):
        print(singer-1,'no singer is being scrapped')
        try: #clicks on singer name
            driver.find_element_by_xpath(f'//*[@id="content"]/div/div/a[{singer}]').click()
            total_songs=[]
            
            total_songs.append(driver.find_elements_by_class_name('onesong'))
            no_of_songs=len(total_songs[0])+1
            
            for elem in range(1,no_of_songs): # this is data downloading block
                com_list=[]
                print('currently scraping song no:',elem, 'out of',no_of_songs-1)
                try:
                    
                    driver.find_element_by_xpath('//*[@id="content"]/div/div/div[{}]/a[2]'.format(elem)).click()
                    
                    movie=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[1]/a').text
                    if not movie:
                        com_list.append(np.nan)
                    else:
                        com_list.append(movie)
                    
                    song=driver.find_element_by_xpath('//*[@id="content"]/div/div/h1').text
                    if not song:
                        com_list.append(np.nan)
                    else:
                        com_list.append(song)
                    singers=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[2]').text
                    if not singers:
                        com_list.append(np.nan)
                    else:
                        com_list.append(singers)
                        
                    lyricist=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[4]').text
                    if not lyricist:
                        com_list.append(np.nan)
                    else:
                        com_list.append(lyricist)
                        
                    year=driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[6]/a[1]').text
                    if not year:
                        com_list.append(np.nan)
                    else:
                        com_list.append(year)
                    try:    
                        com_list.append(driver.find_element_by_xpath('//*[@id="content"]/div/div/ul/li[3]/a').text)
                    except NoSuchElementException:
                        com_list.append(np.nan)
                        
                    lyrics=driver.find_element_by_xpath('//*[@id="content"]/div/div/div[1]').text
                    if not lyrics:
                        com_list.append(np.nan)
                    else:
                        com_list.append(lyrics)
                    
                    temp_df = pd.DataFrame(com_list).values.reshape(1,-1) 
                    xx=pd.DataFrame(temp_df,columns=columns)
                    
                    songs_df_main=pd.concat([songs_df_main,xx], ignore_index=True)
                    #songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # Enable this to save songs in csv as they iterate.
                    
                    driver.back()
                except NoSuchElementException:
                    
                    print("oops song doesn't exist.. moving on to next song ")
                    pass
              
                except Exception as e:
                    print(e)
                    pass
                
            driver.back() 
            
        except ElementNotInteractableException:
          #songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # dumps data to csv after finishing fetching of each letter 
          print("oops singer not clickble.. moving on to next singer ") 
          pass
            
        except Exception as e:
          #songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # dumps data to csv after finishing fetching of each letter 
          print("oops singer doesn't exist.. moving on to next page ") 
          print(e)
          break
          
          
        
        finally :
          songs_df_main.to_csv(f'songs data_{letter}.csv',encoding='utf-8') # dumps data to csv after finishing fetching of each letter


print('All done ..phew :)')    

currently scraping singers by letter k
1 no singer is being scrapped
currently scraping song no: 1 out of 36
currently scraping song no: 2 out of 36
currently scraping song no: 3 out of 36
currently scraping song no: 4 out of 36
currently scraping song no: 5 out of 36
currently scraping song no: 6 out of 36
currently scraping song no: 7 out of 36
currently scraping song no: 8 out of 36
currently scraping song no: 9 out of 36
currently scraping song no: 10 out of 36
currently scraping song no: 11 out of 36
currently scraping song no: 12 out of 36
currently scraping song no: 13 out of 36
currently scraping song no: 14 out of 36
currently scraping song no: 15 out of 36
currently scraping song no: 16 out of 36
currently scraping song no: 17 out of 36
currently scraping song no: 18 out of 36
currently scraping song no: 19 out of 36
currently scraping song no: 20 out of 36
currently scraping song no: 21 out of 36
currently scraping song no: 22 out of 36
currently scraping song no: 23 out of 